In [1]:
import numpy as np
import tensorflow as tf 
from collections import deque
import random
import pdb
from replay_memory import ReplayMemory # AC replay memory
from AC_networks_tf import ActorNet, CriticNet
from test_game import test_env
import os
from __future__ import print_function
from info_obj import info_holder

In [2]:
import gym
game_name = 'LunarLander-v2'
#game_name = 'CartPole-v0'
#game_name = 'MountainCar-v0'
game = gym.make(game_name)
state_dim = game.observation_space.shape[0]
action_dim = game.action_space.n
'''
game = test_env()
state_dim = 1
action_dim = 2
'''

[2016-12-31 23:13:14,332] Making new env: LunarLander-v2


'\ngame = test_env()\nstate_dim = 1\naction_dim = 2\n'

In [3]:
# info = graph, dim_s, dim_a, hid_layers, learning_rate, tau
save_dir = './Results/'+ game_name[:-3] + '/net/'
if 'LunarLander' in game_name:
    LEARNING_RATE = float(1e-2) #5e-5 lr/2
    lr_decay_rate = 1
    C_update = 1 #episode
    GAMMA = 0.99
    MAX_EPISODE = int(1e4)
    MAX_STEP = int(1e3)
    lr_decay_ep = 100
    print_ep = int(1e2)
    save_ep = int(1e3)
    TAU = 0
    ENTROPY_BETA = 0
    
elif 'CartPole' in game_name:
    LEARNING_RATE = 1e-3 #5e-5 lr/2
    lr_decay_rate = 1
    C_update = 1 #episode
    GAMMA = 0.99
    MAX_EPISODE = int(1e4)
    MAX_STEP = int(2e2)
    lr_decay_ep = 100
    print_ep = int(1e2)
    save_ep = int(1e3)
    TAU = 0
    ENTROPY_BETA = 0
    
elif 'MountainCar' in game_name:
    LEARNING_RATE = 1e-3 #5e-5 lr/2
    lr_decay_rate = 0.955
    C_update = 1 #episode
    GAMMA = 0.99
    MAX_EPISODE = int(1e4)
    MAX_STEP = int(2e2)
    lr_decay_ep = 100
    print_ep = int(1e2)
    save_ep = int(1e3)
    TAU = 0
    ENTROPY_BETA = 0
#### load the network
if_load = 'y'
logging_mode = 'w'
strings = ['learning_rate = ', 'lr decay = ', 'update cycle: ']
if 'y' == if_load:
    if not(os.path.isdir(save_dir)):
        raise NameError('No network to upload')
    logging_mode = 'a'
    with open(save_dir + 'params.txt', 'r') as f:
        while True:
            xa = f.readline()
            if strings[0] in xa:
                LEARNING_RATE = int(xa[len(string[0]):])
            elif strings[1] in xa:
                lr_decay_rate = int(xa[len(string[1]):])
            elif strings[2] in xa:
                C_update = int(xa[len(string[23]):])
            break
    with open(save_dir + 'rewards.txt', 'r') as f:
        xa = f.readline()
        xa = xa.split(',')
        start_ep = int(xa[0]) + 1
    var = raw_input("From what episode?(Enter a number or last): ")
    if var != 'last': 
        var_int = int(var)
        while ((var_int%save_ep)!=0) or (var_int > start_ep):
            print('Episode not found!')
            var = raw_input("From what episode?(Enter a number or last): ")
    print('Loading the previous trained network and continue training from episode %d' %start_ep)
else:
    start_ep = 0
lr = LEARNING_RATE * (lr_decay_rate ** (start_ep/lr_decay_ep))# / MINIBATCH_SIZE
#### till here

buff = ReplayMemory()
info = info_holder()
try:
    actor
except:
    pass
else:
    actor.reset_net()
try:
    critic
except:
    pass
else:
    critic.reset_net()  
    
graph = tf.Graph()
with graph.as_default():
    info.graph = graph
    info.dim_s = state_dim
    info.dim_a = action_dim
    info.act_hid_layers = np.array([32])
    info.crit_hid_layers = np.array([16])
    info.learning_rate = LEARNING_RATE
    info.tau = TAU
    info.entropy_beta = ENTROPY_BETA
    actor = ActorNet(info)
    critic = CriticNet(info)

From what episode?(Enter a number or last): last
Loading the previous trained network and continue training from episode 7000


In [4]:
# lr = LEARNING_RATE
buff.clear()
###
last_ep = 0
start_update = 0
with tf.Session(graph=graph) as sess:
    actor.set_up(sess)
    critic.set_up(sess)
    sess.run(tf.global_variables_initializer())
    
    if if_load == 'y':
        saver = tf.train.Saver()
        filename = save_dir + 'Episod_' + str(start_ep-1) + '.chk'
        saver.restore(sess, filename)
    
    ep_reward = np.zeros(MAX_EPISODE)
    ep_success = np.zeros(MAX_EPISODE)
    ep_crash = np.zeros(MAX_EPISODE)
    ep_var = np.zeros(MAX_EPISODE)
    avg_var = 0
    
    for ep in range(MAX_EPISODE):
        s = game.reset()
        var = 0
        for st in range(MAX_STEP):
            a = actor.take_action(s.reshape(-1,info.dim_s))
            #print(actor.give_policy(s.reshape(-1,info.dim_s)))
            s2, r, terminal, report = game.step(a)
            
            buff.add(np.reshape(s, (info.dim_s,)).astype(np.float32), np.array(a).astype(np.float32),\
                     np.array(r).astype(np.float32), np.reshape(s2, (info.dim_s,)).astype(np.float32),\
                     np.array(terminal).astype(np.float32))

            ep_reward[ep] += (1.*r)
            var += np.sum(actor.give_policy(s.reshape(-1,info.dim_s))**2)
            
            s = s2

            if terminal:
                ep_var[ep] = var/st
                #avg_var += (var/print_ep)
                break
        
        if ((ep % C_update) == (C_update -1)):
            (s_batch, a_batch, r_batch, s2_batch, t_batch) = \
                buff.sample_batch()
            s_batch = np.array(s_batch).reshape(-1, info.dim_s)
            s2_batch = np.array(s2_batch).reshape(-1, info.dim_s)
            a_batch = np.array(a_batch)#.reshape(-1, 1)
            r_batch = np.array(r_batch)#.reshape(-1, 1)
            t_batch = np.array(t_batch)#.reshape(-1, 1)
    
            Q_2 = critic.predict(s2_batch)
            Q_2 = Q_2[:,0]
            
            R_batch = r_batch + GAMMA * Q_2 * (1-t_batch)
            R_batch = R_batch.reshape(-1, 1)
            
            predicted_Q, _, _ = critic.train(s_batch, R_batch, lr)
            td_batch = R_batch - predicted_Q
            #print(actor.give_policy(s_batch)[0:9,:])
            #print(td_batch[0:9])
            #print(a_batch[0:9])
            actor.train(s_batch, a_batch, td_batch, lr)
            #print(actor.give_policy(s_batch)[0:9,:])
            #pdb.set_trace()
         
        if terminal == False:
            ep_var[ep] = var/MAX_STEP
            #avg_var += (var/print_ep)
        if (r >= 100):
            ep_success[ep] = 1
        elif (r <= -100):
            ep_crash[ep] = 1
        
        if ((ep % print_ep) == (print_ep-1)) or (ep == 0):                                                      ####
            print('Ep: %d --- Ave R: %.1f  |  Suc: %.2f  |  crash: %.2f  |  Max R: %.1f  |  var: %.2f  |  lr:%.2fe-4'\
                  %((start_ep+ep), np.mean(ep_reward[last_ep:ep+1]), np.mean(ep_success[last_ep:ep+1]), \
                    np.mean(ep_crash[last_ep:ep+1]), np.amax(ep_reward[last_ep:ep+1]), np.mean(ep_var[last_ep:ep+1]), 1e4*lr))
            last_ep = ep
            #avg_var = 0
        
        if (ep == 0) and (if_load == 'n'):  ####
            if (os.path.isdir(save_dir)):
                raise NameError('Directory net exists')
                
        if (ep == (save_ep-1)) and (if_load == 'n'):  ####
            os.mkdir(save_dir)
            with open(save_dir + 'params.txt', 'w') as f:
                f.write('Learning rate = %f\n' %LEARNING_RATE)
                f.write('lr decay = %f\n' %lr_decay_rate)
                f.write('hidden layers: %d\n' %info.hid_layers)
                f.write('update cycle: %d\n' %C_update)
                
        if (ep % save_ep) == (save_ep-1):
            print(' ***** Saving...  at episode %d ...' %(start_ep+ep), end="")                                     ####
            saver = tf.train.Saver()
            filename = save_dir + 'Episod_' + str(start_ep+ep) + '.chk'                                           ####
            saver.save(sess, filename)
            with open(save_dir + 'rewards.txt', logging_mode) as f:                                               ####
                f.write('%d, , , , ,Learning rate = %f - lr decay = %f - C_update = %d - hidden layers: %d\n' \
                        %((start_ep+ep), LEARNING_RATE, lr_decay_rate, C_update, info.hid_layers) )                 ####
                for i in range(ep):
                    f.write('%f,     %f\n' %(ep_reward[i],ep_var[i]))
            print(' Done ******')
            
        if (ep % lr_decay_ep) == (lr_decay_ep-1):
            lr = lr_decay_rate * lr
    
print('DONE') 
    

Ep: 7000 --- Ave R: -151.8  |  Suc: 0.00  |  crash: 1.00  |  Max R: -151.8  |  var: 0.49  |  lr:100.00e-4
Ep: 7099 --- Ave R: -81.2  |  Suc: 0.00  |  crash: 0.30  |  Max R: 31.5  |  var: 0.45  |  lr:100.00e-4
Ep: 7199 --- Ave R: -75.9  |  Suc: 0.00  |  crash: 0.33  |  Max R: 18.1  |  var: 0.45  |  lr:100.00e-4
Ep: 7299 --- Ave R: -74.1  |  Suc: 0.00  |  crash: 0.24  |  Max R: 40.9  |  var: 0.45  |  lr:100.00e-4
Ep: 7399 --- Ave R: -72.1  |  Suc: 0.00  |  crash: 0.30  |  Max R: 53.0  |  var: 0.45  |  lr:100.00e-4
Ep: 7499 --- Ave R: -72.7  |  Suc: 0.01  |  crash: 0.29  |  Max R: 131.9  |  var: 0.46  |  lr:100.00e-4
Ep: 7599 --- Ave R: -59.0  |  Suc: 0.02  |  crash: 0.14  |  Max R: 76.4  |  var: 0.46  |  lr:100.00e-4
Ep: 7699 --- Ave R: -37.3  |  Suc: 0.01  |  crash: 0.18  |  Max R: 101.3  |  var: 0.46  |  lr:100.00e-4
Ep: 7799 --- Ave R: -31.9  |  Suc: 0.02  |  crash: 0.14  |  Max R: 132.7  |  var: 0.46  |  lr:100.00e-4
Ep: 7899 --- Ave R: -25.5  |  Suc: 0.06  |  crash: 0.16  |  Max R: 

KeyboardInterrupt: 

sess.run(actor.responsible_outputs, feed_dict={actor.inputs:s_batch, actor.taken_action:a_batch})
sess.run(actor.log_pi, feed_dict={actor.inputs:s_batch, actor.taken_action:a_batch})
td_batch
xx = sess.run(actor.log_pi, feed_dict={actor.inputs:s_batch, actor.taken_action:a_batch})*td_batch
xx = sess.run(actor.loss_vector, feed_dict={actor.inputs:s_batch, actor.taken_action:a_batch, actor.td:td_batch})
xx.shape
actor.give_policy(s_batch)

## 
##
##
##
##
##
##
##
##
##
##
##
##
##
##
##

In [ ]:
print('Ep: %d --- Ave R: %.1f  |  Suc: %.2f  |  crash: %.2f  |  Max R: %.1f  |  var: %.2f  |  lr:%.2fe-4'\
                  %((start_ep+ep), np.mean(ep_reward[last_ep:ep+1]), np.mean(ep_success[last_ep:ep+1]), \
                    np.mean(ep_crash[last_ep:ep+1]), np.amax(ep_reward[last_ep:ep+1]), np.mean(ep_var[last_ep:ep+1]), 1e4*lr))

In [ ]:
last_ep = 0